# Advanced Model Integration Tutorial

This tutorial covers advanced techniques for integrating multiple AI models and providers with the Orchestrator framework.

## Overview

The Orchestrator framework supports:

- **Multiple Providers**: OpenAI, Anthropic, Google, local models
- **Dynamic Model Selection**: Automatic best-fit model selection
- **Model Capabilities**: Understanding what each model can do
- **Fallback Strategies**: Automatic failover between models
- **Cost Optimization**: Choosing models based on cost and performance
- **Streaming Support**: Real-time response streaming

## Setup

Let's start by importing the necessary components:

In [ ]:
import sys
sys.path.insert(0, '../src')

import os
import json
from typing import Dict, List, Any

from orchestrator.models.openai_model import OpenAIModel
from orchestrator.models.anthropic_model import AnthropicModel
from orchestrator.utils.api_keys import load_api_keys
from orchestrator.models.model_registry import ModelRegistry
from orchestrator.core.task import Task
from orchestrator.core.pipeline import Pipeline
from orchestrator.orchestrator import Orchestrator
from orchestrator.state.state_manager import InMemoryStateManager

# Load API keys
load_api_keys()

print("✅ Advanced model integration components imported!")

## Working with Multiple Real Models

Let's set up different models from various providers:

In [ ]:
# Create models from different providers
# Note: You need to have the appropriate API keys set as environment variables

# OpenAI Models
gpt4_model = OpenAIModel(
    name="gpt-4",
    api_key=os.environ.get("OPENAI_API_KEY"),
)

gpt35_model = OpenAIModel(
    name="gpt-3.5-turbo",
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Anthropic Model (if you have an API key)
if os.environ.get("ANTHROPIC_API_KEY"):
    claude_model = AnthropicModel(
        name="claude-2",
        api_key=os.environ.get("ANTHROPIC_API_KEY"),
    )
    print("✅ Anthropic Claude model configured")
else:
    claude_model = None
    print("ℹ️ Anthropic API key not found, skipping Claude model")

print(f"✅ OpenAI models configured: GPT-4 and GPT-3.5-turbo")

## Creating a Model Registry

The model registry helps manage multiple models and their selection:

In [ ]:
# Create model registry and register models
registry = ModelRegistry()

# Register OpenAI models
registry.register_model(gpt4_model)
registry.register_model(gpt35_model)

# Register Anthropic model if available
if claude_model:
    registry.register_model(claude_model)

print(f"🗂️ Model Registry created with {len(registry.list_models())} models:")
for model_name in registry.list_models():
    model = registry.get_model(model_name)
    print(f"   📱 {model.name} - Ready for use")

## Model Selection Based on Task Requirements

Different models have different strengths. Let's create tasks that benefit from specific models:

In [ ]:
# Create a pipeline with model-specific tasks
multi_model_pipeline = Pipeline(
    id="multi_model_demo",
    name="Multi-Model Demonstration Pipeline",
    description="Demonstrates using different models for different tasks"
)

# Task 1: Complex reasoning (best suited for GPT-4)
reasoning_task = Task(
    id="complex_reasoning",
    name="Complex Reasoning Task",
    action="generate",
    parameters={
        "prompt": "Solve this logic puzzle: Three friends (Alice, Bob, Charlie) have different colored hats (red, blue, green). Alice's hat is not red. Bob's hat is not green. Charlie's hat is not blue. If Alice's hat is green, what color is each person's hat?",
        "max_tokens": 200
    },
    metadata={
        "preferred_model": "gpt-4",  # Complex reasoning benefits from GPT-4
        "requirements": {
            "reasoning_capability": "high"
        }
    }
)

# Task 2: Quick summary (efficient with GPT-3.5)
summary_task = Task(
    id="quick_summary",
    name="Quick Summary",
    action="generate",
    parameters={
        "prompt": "Summarize the solution in one sentence: {complex_reasoning}",
        "max_tokens": 50
    },
    dependencies=["complex_reasoning"],
    metadata={
        "preferred_model": "gpt-3.5-turbo",  # Simple task, use efficient model
        "requirements": {
            "speed": "fast",
            "cost": "low"
        }
    }
)

# Task 3: Creative expansion (could use any capable model)
creative_task = Task(
    id="creative_expansion",
    name="Creative Expansion",
    action="generate",
    parameters={
        "prompt": "Write a short creative story about people wearing the hats from this puzzle: {complex_reasoning}",
        "max_tokens": 300
    },
    dependencies=["complex_reasoning"],
    metadata={
        "preferred_model": "any",  # Any model can handle creative tasks
        "requirements": {
            "creativity": "high"
        }
    }
)

# Add tasks to pipeline
multi_model_pipeline.add_task(reasoning_task)
multi_model_pipeline.add_task(summary_task)
multi_model_pipeline.add_task(creative_task)

print(f"🧠 Multi-model pipeline created with {len(multi_model_pipeline)} tasks")
print(f"📋 Execution order: {multi_model_pipeline.get_execution_order()}")

## Smart Orchestrator with Model Assignment

Create an orchestrator that can intelligently assign models to tasks:

In [ ]:
class SmartOrchestrator(Orchestrator):
    """Enhanced orchestrator with intelligent model assignment."""
    
    def __init__(self, state_manager, model_registry: ModelRegistry):
        super().__init__(state_manager)
        self.model_registry = model_registry
        
        # Register all models from registry
        for model_name in model_registry.list_models():
            model = model_registry.get_model(model_name)
            self.register_model(model)
    
    def select_model_for_task(self, task: Task) -> str:
        """Select the best model for a given task."""
        # Check if task has a preferred model
        preferred = task.metadata.get("preferred_model")
        
        if preferred and preferred != "any":
            # Use preferred model if available
            if preferred in self.models:
                return preferred
        
        # Otherwise, select based on requirements
        requirements = task.metadata.get("requirements", {})
        
        # Simple heuristic for model selection
        if requirements.get("reasoning_capability") == "high":
            return "gpt-4" if "gpt-4" in self.models else "gpt-3.5-turbo"
        elif requirements.get("speed") == "fast" or requirements.get("cost") == "low":
            return "gpt-3.5-turbo"
        else:
            # Default to first available model
            return list(self.models.keys())[0]
    
    async def execute_pipeline_with_smart_assignment(self, pipeline: Pipeline) -> Dict[str, Any]:
        """Execute pipeline with automatic model assignment."""
        results = {}
        
        print(f"🧠 Executing pipeline with smart model assignment\n")
        
        for task_id in pipeline.get_execution_order()[0]:  # Simple sequential execution
            task = pipeline.get_task(task_id)
            
            # Select best model for this task
            selected_model_name = self.select_model_for_task(task)
            selected_model = self.models[selected_model_name]
            
            print(f"📋 Task: {task.name}")
            print(f"   🤖 Selected Model: {selected_model_name}")
            
            # Execute task
            task.start()
            
            try:
                # Prepare prompt with dependencies
                prompt = task.parameters["prompt"]
                for dep in task.dependencies:
                    if dep in results:
                        prompt = prompt.replace(f"{{{dep}}}", results[dep])
                
                # Execute with selected model
                result = await selected_model.generate(
                    prompt,
                    max_tokens=task.parameters.get("max_tokens", 100)
                )
                
                task.complete(result)
                results[task_id] = result
                print(f"   ✅ Completed successfully\n")
                
            except Exception as e:
                task.fail(e)
                print(f"   ❌ Failed: {e}\n")
                raise
        
        return results

# Create smart orchestrator
state_manager = InMemoryStateManager()
smart_orchestrator = SmartOrchestrator(state_manager, registry)

print(f"🧠 Smart orchestrator created with {len(smart_orchestrator.models)} models")
print(f"📊 Available models: {list(smart_orchestrator.models.keys())}")

## Execute the Multi-Model Pipeline

Now let's run our pipeline with real AI models:

In [ ]:
async def run_multi_model_pipeline():
    """Execute the multi-model pipeline."""
    print("🚀 Starting multi-model pipeline execution...\n")
    
    try:
        # Execute with smart model assignment
        results = await smart_orchestrator.execute_pipeline_with_smart_assignment(multi_model_pipeline)
        
        print("\n📊 Pipeline Results:")
        print("=" * 50)
        
        for task_id, result in results.items():
            task = multi_model_pipeline.get_task(task_id)
            print(f"\n🔸 {task.name}:")
            print(f"{result}")
        
        return results
        
    except Exception as e:
        print(f"❌ Pipeline execution failed: {e}")
        return None

# Execute the pipeline
# Note: This will make real API calls to OpenAI/Anthropic
results = await run_multi_model_pipeline()

## Model Performance Comparison

Let's compare how different models perform on the same task:

In [ ]:
async def compare_models_on_task(prompt: str, max_tokens: int = 100):
    """Compare different models on the same task."""
    print(f"🔬 Comparing models on task:\n'{prompt}'\n")
    
    comparison_results = {}
    
    for model_name in smart_orchestrator.models:
        model = smart_orchestrator.models[model_name]
        print(f"Testing {model_name}...")
        
        try:
            import time
            start_time = time.time()
            
            result = await model.generate(prompt, max_tokens=max_tokens)
            
            end_time = time.time()
            execution_time = end_time - start_time
            
            comparison_results[model_name] = {
                "response": result,
                "execution_time": execution_time,
                "tokens_approx": len(result.split())  # Rough approximation
            }
            
        except Exception as e:
            comparison_results[model_name] = {
                "error": str(e)
            }
    
    # Display results
    print("\n📊 Comparison Results:")
    print("=" * 50)
    
    for model_name, result in comparison_results.items():
        print(f"\n🤖 {model_name}:")
        if "error" in result:
            print(f"   ❌ Error: {result['error']}")
        else:
            print(f"   ⏱️ Time: {result['execution_time']:.2f}s")
            print(f"   📝 Tokens (approx): {result['tokens_approx']}")
            print(f"   Response: {result['response'][:200]}..." if len(result['response']) > 200 else f"   Response: {result['response']}")
    
    return comparison_results

# Compare models on a specific task
comparison_prompt = "Explain the concept of recursion in programming using a simple analogy."
comparison = await compare_models_on_task(comparison_prompt, max_tokens=150)

## Cost-Aware Pipeline Execution

Monitor and optimize costs across different models:

In [ ]:
# Define approximate costs per 1K tokens (example values)
MODEL_COSTS = {
    "gpt-4": {"input": 0.03, "output": 0.06},
    "gpt-3.5-turbo": {"input": 0.001, "output": 0.002},
    "claude-2": {"input": 0.008, "output": 0.024}
}

def estimate_cost(model_name: str, input_tokens: int, output_tokens: int) -> float:
    """Estimate cost for a model execution."""
    if model_name not in MODEL_COSTS:
        return 0.0
    
    costs = MODEL_COSTS[model_name]
    input_cost = (input_tokens / 1000) * costs["input"]
    output_cost = (output_tokens / 1000) * costs["output"]
    
    return input_cost + output_cost

# Create a cost-optimized pipeline
cost_pipeline = Pipeline(
    id="cost_optimized",
    name="Cost-Optimized Pipeline",
    description="Demonstrates cost-aware model selection"
)

# Mix of tasks with different cost sensitivities
tasks = [
    Task(
        id="expensive_analysis",
        name="Complex Analysis (Worth GPT-4)",
        action="generate",
        parameters={
            "prompt": "Analyze the economic implications of quantum computing on cybersecurity markets over the next decade.",
            "max_tokens": 300
        },
        metadata={"cost_sensitivity": "low", "quality_requirement": "high"}
    ),
    Task(
        id="simple_summary",
        name="Simple Summary (Use Efficient Model)",
        action="generate",
        parameters={
            "prompt": "List three key points from: {expensive_analysis}",
            "max_tokens": 100
        },
        dependencies=["expensive_analysis"],
        metadata={"cost_sensitivity": "high", "quality_requirement": "medium"}
    ),
    Task(
        id="translation",
        name="Translation (Any Model Works)",
        action="generate",
        parameters={
            "prompt": "Translate to Spanish: {simple_summary}",
            "max_tokens": 150
        },
        dependencies=["simple_summary"],
        metadata={"cost_sensitivity": "medium", "quality_requirement": "medium"}
    )
]

for task in tasks:
    cost_pipeline.add_task(task)

print("💰 Cost-optimized pipeline created")
print("📊 Tasks with cost sensitivities:")
for task in tasks:
    print(f"   - {task.name}: {task.metadata['cost_sensitivity']} cost sensitivity")

## Model Fallback and Error Handling

Implement robust error handling with model fallbacks:

In [ ]:
class RobustOrchestrator(SmartOrchestrator):
    """Orchestrator with fallback capabilities."""
    
    async def execute_with_fallback(self, task: Task, primary_model: str) -> Any:
        """Execute task with fallback to other models if primary fails."""
        models_to_try = [primary_model]
        
        # Add other models as fallbacks
        for model_name in self.models:
            if model_name != primary_model:
                models_to_try.append(model_name)
        
        last_error = None
        
        for model_name in models_to_try:
            try:
                print(f"   🔄 Trying {model_name}...")
                model = self.models[model_name]
                
                result = await model.generate(
                    task.parameters["prompt"],
                    max_tokens=task.parameters.get("max_tokens", 100)
                )
                
                print(f"   ✅ Success with {model_name}")
                return result
                
            except Exception as e:
                last_error = e
                print(f"   ⚠️ Failed with {model_name}: {e}")
                continue
        
        # All models failed
        raise Exception(f"All models failed. Last error: {last_error}")

# Create robust orchestrator
robust_orchestrator = RobustOrchestrator(state_manager, registry)

print("🛡️ Robust orchestrator created with fallback capabilities")

## Summary

In this tutorial, you learned:

1. **Real Model Integration**: Working with actual AI providers (OpenAI, Anthropic)
2. **Model Registry**: Managing multiple models in a unified way
3. **Smart Model Selection**: Choosing the right model for each task
4. **Cost Optimization**: Balancing quality and cost across models
5. **Performance Comparison**: Evaluating different models on the same tasks
6. **Fallback Strategies**: Ensuring reliability with model fallbacks
7. **Error Handling**: Graceful degradation when models fail

## Key Takeaways

- **🎯 Match Models to Tasks**: Use powerful models for complex tasks, efficient models for simple ones
- **💰 Monitor Costs**: Track API usage and optimize model selection
- **🔄 Build Resilience**: Always have fallback options
- **📊 Measure Performance**: Compare models to make informed decisions
- **🧠 Smart Assignment**: Let the orchestrator choose models based on requirements

## Best Practices

1. **Always Use Real API Keys**: Set up environment variables for each provider
2. **Start Small**: Test with smaller models before using expensive ones
3. **Monitor Usage**: Keep track of API calls and costs
4. **Handle Errors**: Implement proper error handling and retries
5. **Cache Results**: Avoid redundant API calls when possible
6. **Set Limits**: Use max_tokens and other parameters to control costs

## Next Steps

- Explore **Streaming Responses** for real-time generation
- Learn about **Structured Output** with JSON schemas
- Try **Fine-tuned Models** for specialized tasks
- Implement **Custom Model Adapters** for proprietary models
- Set up **Production Monitoring** for model performance

---

**Happy orchestrating with real AI models! 🚀🤖**